In [1]:
from utils import train_model, unfreeze_layers, get_resnet_model, get_efficientnet_model, validate_model, \
    get_xception_model, get_vgg_model, get_resnetv2_model, get_nasnetmobile_model, get_mobilenetv2_model, \
    get_mobilenetv1_model, get_inception_model, get_inceptionresnet_model

2022-04-08 12:38:39.299570: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2022-04-08 12:38:39.299776: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [41]:
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import GlobalAveragePooling2D, BatchNormalization, Dropout, Dense, Flatten
from tensorflow.keras import Model



def get_mobilenetv1_model():
    base_model = MobileNet(include_top=True, weights='imagenet', input_shape=(400, 300, 3))
    top_model = GlobalAveragePooling2D(name="avg_pool")(base_model.output)
    top_model = layers.Reshape((1, 1, 1024), name='reshape_1')(top_model)
    top_model = layers.Dropout(0.2, name='dropout')(top_model)
    top_model = layers.Conv2D(classes, (1, 1), padding='same', name='conv_preds')(top_model)
    top_model = layers.Reshape((6,), name='reshape_2')(top_model)
    top_model = Dense(6, activation='softmax', name='diagnosis')(top_model)
    return Model(inputs=base_model.input, outputs=top_model, name="MobileNetV1")

In [40]:
mobilenetv1 = get_mobilenetv1_model()
mobilenetv1.summary()

ValueError: Input 0 of layer avg_pool is incompatible with the layer: expected ndim=4, found ndim=2. Full shape received: [None, 1000]

In [22]:
inceptionresnet = get_inceptionresnet_model()
inceptionresnet.summary()

Model: "InceptionResNetV2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           [(None, 400, 300, 3) 0                                            
__________________________________________________________________________________________________
conv2d_1362 (Conv2D)            (None, 199, 149, 32) 864         input_10[0][0]                   
__________________________________________________________________________________________________
batch_normalization_1363 (Batch (None, 199, 149, 32) 96          conv2d_1362[0][0]                
__________________________________________________________________________________________________
activation_1362 (Activation)    (None, 199, 149, 32) 0           batch_normalization_1363[0][0]   
__________________________________________________________________________________

In [3]:
efficientnet = get_efficientnet_model()
efficientnet.summary()

16711680/16705208 [==============================] - 1s 0us/step
Model: "EfficientNetB0"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 400, 300, 3) 0                                            
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, 400, 300, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
normalization (Normalization)   (None, 400, 300, 3)  7           rescaling[0][0]                  
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 401, 301, 3)  0           normalization[0][0]              
____________________

block5c_se_excite (Multiply)    (None, 25, 19, 672)  0           block5c_activation[0][0]         
                                                                 block5c_se_expand[0][0]          
__________________________________________________________________________________________________
block5c_project_conv (Conv2D)   (None, 25, 19, 112)  75264       block5c_se_excite[0][0]          
__________________________________________________________________________________________________
block5c_project_bn (BatchNormal (None, 25, 19, 112)  448         block5c_project_conv[0][0]       
__________________________________________________________________________________________________
block5c_drop (Dropout)          (None, 25, 19, 112)  0           block5c_project_bn[0][0]         
__________________________________________________________________________________________________
block5c_add (Add)               (None, 25, 19, 112)  0           block5c_drop[0][0]               
          

In [2]:
resnet = get_resnet_model()
resnet.summary()

2022-04-08 12:39:08.652490: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-04-08 12:39:08.652535: W tensorflow/stream_executor/cuda/cuda_driver.cc:312] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-08 12:39:08.652729: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (yoga9): /proc/driver/nvidia/version does not exist
2022-04-08 12:39:08.653085: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-08 12:39:08.669269: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Frequency: 2592000000 Hz
20

94773248/94765736 [==============================] - 4s 0us/step
Model: "ResNet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 400, 300, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 406, 306, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 200, 150, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 200, 150, 64) 256         conv1_conv[0][0]                 
__________________________

In [ ]:
import glob
import pandas as pd
from pathlib import Path
from sklearn.metrics import classification_report

base_path = "/home/ubuntu/store/efficientnet-hpo"
model_preds = glob.glob("/home/ubuntu/store/efficientnet-hpo/*_preds.csv")
model_comparison_dict = {}

for model_pred in model_preds:
    model_preds_df = pd.read_pickle(Path(model_pred))
    model_comparison_dict[Path(model_pred).stem] = classification_report(
        model_preds_df['actual'],
        model_preds_df['pred'],
        labels=[0, 1, 2, 3, 4, 5],
        target_names=['acne', 'actinic_keratosis', 'psoriasis_no_pustular', 'seborrheic_dermatitis', 'vitiligo', 'wart'],
        output_dict=True
    )['macro avg']

model_comparison_df = pd.DataFrame.from_dict(model_comparison_dict, orient='index')
model_comparison_df